In [1]:
import xarray as xr # for netCDF4 
import pandas as pd
import numpy as np

# Open nc file

When you open the file without a group defined, you get the global attributes with no variables. You need to include a group='PRODUCT' to get the data product.

Beaware that if you do not know the group, you cannot use xarray. Instead use netCDF4.

In [2]:
from netCDF4 import Dataset
file_path = r"C:\\Users\\joonw\\Downloads\\GK2_GEMS_L2_20240308_2345_O3T_HE_DPRO_ORI.nc"
rootgrp = Dataset(file_path, "r")
print(rootgrp.groups)

{'Data Fields': <class 'netCDF4._netCDF4.Group'>
group /Data Fields:
    dimensions(sizes): 
    variables(dimensions): float32 AveragingKernel(nlayer2, nlayer2, spatial, image), float32 CloudPressure(spatial, image), float32 ColumnAmountO3(spatial, image), float32 DegreesOfFreedomForSignal(spatial, image), float32 EstimatedError(spatial, image), int16 FinalAlgorithmFlags(spatial, image), float32 LayerEfficiency(nlayer2, spatial, image), float32 Nvalue(spatial, image, nwl), float32 O3BelowCloud(spatial, image), float32 Reflectivity340(spatial, image), float32 Reflectivity380(spatial, image), float32 Residue(spatial, image, nsignal), float32 StepOneO3(spatial, image), float32 StepTwoO3(spatial, image), float32 TerrainPressure(spatial, image), float32 dNdR(spatial, image, dim_reflectivity), float32 dR_dl(spatial, image), float32 EffectiveCloudFraction(spatial, image)
    groups: , 'Geolocation Fields': <class 'netCDF4._netCDF4.Group'>
group /Geolocation Fields:
    dimensions(sizes): 
  

In [3]:
file_path = r"C:\\Users\\joonw\\Downloads\\TCO_analysis\\20230209T.nc\S5P_NRTI_L2__O3_____20230209T.nc"

ds = xr.open_dataset(file_path, group='PRODUCT')
# Close the NetCDF file

ds.head()
# ds.close()
ds.info()



xarray.Dataset {
dimensions:
	scanline = 357 ;
	ground_pixel = 450 ;
	time = 1 ;
	corner = 4 ;
	layer = 13 ;
	level = 14 ;

variables:
	float64 scanline(scanline) ;
		scanline:units = 1 ;
		scanline:axis = Y ;
		scanline:long_name = along-track dimension index ;
		scanline:comment = This coordinate variable defines the indices along track; index starts at 0 ;
	float64 ground_pixel(ground_pixel) ;
		ground_pixel:units = 1 ;
		ground_pixel:axis = X ;
		ground_pixel:long_name = across-track dimension index ;
		ground_pixel:comment = This coordinate variable defines the indices across track, from west to east; index starts at 0 ;
	datetime64[ns] time(time) ;
		time:standard_name = time ;
		time:axis = T ;
		time:long_name = reference time for the measurements ;
		time:comment = The time in this variable corresponds to the time in the time_reference global attribute ;
	float64 corner(corner) ;
		corner:units = 1 ;
		corner:long_name = pixel corner index ;
		corner:comment = This coordinate 

In [113]:
dd = ds['time_utc']
dd.to_dataframe().reset_index()


dd = ds['latitude']
dd = dd.to_dataframe().reset_index()

print(dd.iloc[1:5,:])
print(dd.iloc[100:105,:])
print(dd.iloc[200:205,:])
print(dd.iloc[20000:20005,:])


        time  scanline  ground_pixel  latitude
1 2023-02-09       0.0           1.0  6.469116
2 2023-02-09       0.0           2.0  6.501437
3 2023-02-09       0.0           3.0  6.533309
4 2023-02-09       0.0           4.0  6.564741
          time  scanline  ground_pixel  latitude
100 2023-02-09       0.0         100.0  9.228599
101 2023-02-09       0.0         101.0  9.244569
102 2023-02-09       0.0         102.0  9.260383
103 2023-02-09       0.0         103.0  9.276043
104 2023-02-09       0.0         104.0  9.291553
          time  scanline  ground_pixel   latitude
200 2023-02-09       0.0         200.0  10.336202
201 2023-02-09       0.0         201.0  10.344195
202 2023-02-09       0.0         202.0  10.352153
203 2023-02-09       0.0         203.0  10.360074
204 2023-02-09       0.0         204.0  10.367961
            time  scanline  ground_pixel   latitude
20000 2023-02-09      44.0         200.0  12.507050
20001 2023-02-09      44.0         201.0  12.515080
20002 2023-02-0

For now, I want to use 'latitude', 'longitude', 'ozone_total_vertical_column' only.

In [8]:
# Select specific variables
selected_variables = ['latitude', 'longitude', 'delta_time', 'ozone_total_vertical_column']
selected_ds = ds[selected_variables]

# Convert xarray.Dataset to pandas DataFrame
df = selected_ds.to_dataframe().reset_index()
# Somehow this gives 6 columns

df2 = df[['latitude', 'longitude', 'delta_time', 'ozone_total_vertical_column']]

print(df2.shape)

# Many NaNs

df2_cleaned = df2.dropna(subset=['latitude', 'longitude', 'ozone_total_vertical_column'])

# Print the resulting cleaned DataFrame
print(df2_cleaned.shape)

(160650, 4)
(160293, 4)


Now I want to save the dataframe as csv file so chat I can work in R.

In [75]:
import os

# Assuming df2_cleaned is the cleaned DataFrame
directory_path = r"C:\\Users\\joonw\Downloads\\TCO_analysis"
csv_file_path = os.path.join(directory_path, 'df2_cleaned.csv')
# Save the DataFrame to a CSV file in the specified directory
df2_cleaned.to_csv(csv_file_path, index=False)

# How to read HDF file in Python

In [ ]:
import numpy as np
import os
import pyhdf    # hd4
from pyhdf.SD import SD
import pandas as pd
import xarray as xr # for netCDF4 

# Sear surface temperature anomaly 

## Data types

National Environmental Satellite, Data and Information Service(NESDIS)

1) 2000 to 2020: Twice-weekly global 50km resolution HDF data.

Each data correponds to one day (a single observation in a day) and has a shape of (238320, 4) with longitude, latitude and SST_anomaly and surface_flag. 

https://coralreefwatch.noaa.gov/product/50km/index.php

2) 1985 to present: Daily regional 5km resolution NetCDF4 data.

Each data correponds to one day(single observation in a day)  has a shape of (641602, 6),(341902) with  'time', 'longitude', 'latitude', 'sea_surface_temperature_anomaly', 'mask' and 'crs'.     

https://coralreefwatch.noaa.gov/product/5km/index.php

3) 2000 to 2020: 227 stations' time series data recorded twice-weekly. 

Each data contains observations from 2000 to 2020 with fixed longtidue and latitude.


https://coralreefwatch.noaa.gov/product/50km/list_vs_group_latlon_201103.php


#### Extract data from the website 

https://coastwatch.pfeg.noaa.gov/erddap/griddap/NOAA_DHW.html

https://coastwatch.pfeg.noaa.gov/erddap/griddap/NOAA_DHW.csv?CRW_SSTANOMALY%5B(2024-02-1T12:00:00Z):1:(2024-02-1T12:00:00Z)%5D%5B(-20):1:(20)%5D%5B(-20):1:(20)%5D


Terminology:

BAA: bleaching alert area    
mask: pixel characteristics flag    
DHW: degree heating week, Celsius weeks   
HOTSPOT: coral bleaching hotspot, Celsius   
SEAICE: sea ice fraction, 1   

## Check if your HDF file is hdf4 or hdf5

hdf4 format:  b'\x0e\x03\x13\x01'   
hdf5 format:  b'\x89HDF'   


In [ ]:
hdf4_file_path = r"C:\\Users\\joonw\\Downloads\\td1.hdf"
h=open(hdf4_file_path, 'rb'); bts=h.read(4); print(bts)

In [ ]:
hdf_file = SD(hdf4_file_path)

# Print a list of dataset names
datasets = hdf_file.datasets()
print("Available Datasets:")
for dataset_name in datasets:
    print(dataset_name)
    
hdf_file.end()

## Reshape the data

In [ ]:
hdf_file = SD(hdf4_file_path)

longitude_data = hdf_file.select("longitude").get()
latitude_data = hdf_file.select("latitude").get()
wind_data = hdf_file.select("CRW_SSTANOMALY").get()

hdf_file.end()

# print(data1.shape)
# print(data2.shape)
# print(data3.shape)

# Reshape latitude and longitude to match the wind data shape
longitude_data2 = np.tile(longitude_data, (1, len(latitude_data )))
latitude_data2 = np.repeat(latitude_data, len(longitude_data ))            # why longitude ordered - to + and latitude from + to -

# Create a DataFrame
df = pd.DataFrame({
    'Longitude': longitude_data2.flatten(),
    'Latitude': latitude_data2.flatten(),
    'SST_Anomaly': wind_data.flatten()
})

print(df.shape)
df.describe()